# gdgtm regression test

The goal of this test is to evaluate whether the gdgtm (**g**eospatial **d**ata **g**etting, **t**ransforming, **m**anaging) pipeline built to feed standardized rasters for downstream ML actions (overall goal of the gdgtm package) is working as intended. The test script is structured as follows:

- Section 1: Test the flow of the relevant "get" functions
- Section 2: Test the core pipeline for creating the "master" raster
- Section 3: Test the mosaics and raster combination components
- Section 4: Test the "standardization" line
- Section 5: Test the "validation" line
- Section 6: Test the overall pipeline implementation

In [ ]:
import gdgtm
from pathlib import Path

## Section 1: "get" function tests
- Target folder: "/home/pete/Documents/tests_and_vals/gdgtm/get_functions/"

### get_chelsa_data

In [ ]:
### get_chelsa_data (1): happy path, tas parameter

parameter = "tas"
extent = [7.3, 7.5, 47.0, 47.2]
start_date = "2023-1-1"
end_date = "2023-2-2"

gdgtm.get_chelsa_data(parameter, extent, start_date, end_date, 
                      write_location = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_tas_raw.tif')

## Expected outcome: str: 'Target .tiff exists' and file meta


#### Test outcome

In [ ]:
### get_chelsa_data (2): happy path, pr parameter

parameter = "pr"
extent = [7.3, 7.5, 47.0, 47.2]
start_date = "2023-1-1"
end_date = "2023-2-2"

gdgtm.get_chelsa_data(parameter, extent, start_date, end_date, 
                      write_location = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif')

## Expected outcome: str: 'Target .tiff exists' and file meta


#### Test outcome

In [ ]:
### get_chelsa_data (3): error path, made up parameter

parameter = "obviously_wrong_param"
extent = [7.3, 7.5, 47.0, 47.2]
start_date = "2023-1-1"
end_date = "2023-2-2"

gdgtm.get_chelsa_data(parameter, extent, start_date, end_date, 
                      write_location = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif')

## Expected outcome: Error message


#### Test outcome

### get_cognames_from_stac_coll_static

In [ ]:
### get_cognames_from_stac_coll_static (1): correct url for an OLM collection value

test = gdgtm.get_cognames_from_stac_coll_static("https://s3.eu-central-1.wasabisys.com/stac/openlandmap/wilderness_li2022.human.footprint/collection.json")

print(test[0])

## Expected outcome: url to a geotiff

#### Test outcome

In [ ]:
### get_cognames_from_stac_coll_static (2): correct url for a Capella collection value

test = gdgtm.get_cognames_from_stac_coll_static("https://capella-open-data.s3.us-west-2.amazonaws.com/stac/capella-open-data-by-instrument-mode/capella-open-data-stripmap/collection.json")

print(test[0])

## Expected outcome: url to a geotiff

#### Test outcome

In [ ]:
### get_cognames_from_stac_coll_static (3): wrong information provided

test = gdgtm.get_cognames_from_stac_coll_static("https://www.unifr.ch/theo/fr/")

print(test[0])

## Expected outcome: an error message

#### Test outcome

### get_cogs_from_olm

In [ ]:
### get_cogs_from_olm (1): correct url for an OLM collection value - Human footprint

test = gdgtm.get_cognames_from_stac_coll_static("https://s3.eu-central-1.wasabisys.com/stac/openlandmap/wilderness_li2022.human.footprint/collection.json")

bbox = (5.7663, 47.9163, 10.5532, 45.6755)

gdgtm.get_cogs_from_olm(cognames = test, 
                        target_directory = "/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/", 
                        target_names = "olm_humfoot_switz_raw_",
                        bbox = bbox,
                        date_start = "20000601",
                        date_end = "20050101"
                       )

## Expected outcome: a list of four downloaded files

#### Test outcome

In [ ]:
### get_cogs_from_olm (2): correct url for an OLM collection value - ESA CCI annual land cover

test = gdgtm.get_cognames_from_stac_coll_static("https://s3.eu-central-1.wasabisys.com/stac/openlandmap/land.cover_esacci.lc.l4/collection.json")

bbox = (5.7663, 47.9163, 10.5532, 45.6755)

gdgtm.get_cogs_from_olm(cognames = test, 
                        target_directory = "/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/", 
                        target_names = "olm_alc_switz_raw_",
                        bbox = bbox,
                        date_start = "20000601",
                        date_end = "20050101"
                       )

## Expected outcome: a list of four downloaded files

#### Test outcome

In [ ]:
### get_cogs_from_olm (3): test defaults

test = gdgtm.get_cognames_from_stac_coll_static("https://s3.eu-central-1.wasabisys.com/stac/openlandmap/ph.h2o_usda.4c1a2a/collection.json")

gdgtm.get_cogs_from_olm(cognames = test, 
                        target_directory = "/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/", 
                        target_names = "olm_soilph_eurafr_raw_"
                       )

## Expected outcome: a list of one downloaded file

#### Test outcome

## Section 2: creating the "Master" geotiff

### reproject_raster

In [ ]:
### reproject_raster (1): error - provide faulty new_crs

gdgtm.reproject_raster(new_crs = "ggplot:2", 
                       source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif',
                       dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected_error.tif',
                       delete_source = True)

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif'))
## Expected outcome: Error message, no file changes

#### Test outcome

In [ ]:
### reproject_raster (2): happy path - no file deletion

gdgtm.reproject_raster(new_crs = "ggplot:2", 
                       source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif',
                       dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif',
                       delete_source = False)

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif'))
## Expected outcome: new file confirmation message ("Reprojection successful"), confirm that source_file retained

#### Test outcome

In [ ]:
### reproject_raster (3): happy path, delete source by default

gdgtm.reproject_raster(new_crs = "ggplot:2", 
                       source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif',
                       dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif'
                      )

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/01_get_functions/chelsa_pr_raw.tif'))
## Expected outcome: new file confirmation message ("Reprojection successful"), confirm that source_file not retained

#### Test outcome

### change_raster_resolution

In [1]:
### change_raster_resolution (1): failure - provide invalid value for target res

gdgtm.change_raster_res(target_res = "five-hundred",
                        source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif',
                        dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                        delete_source = True
                       )

print("Source file retained: " + Path('home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif'))
## Expected outcome: Error message, file retained

#### Test outcome

In [3]:
### change_raster_resolution (2): happy path - retain source raster

gdgtm.change_raster_res(target_res = 500,
                        source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif',
                        dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                        delete_source = False
                       )

print("Source file retained: " + Path('home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif'))
## Expected outcome: Match confirmation ("Resolution change successful: new pixel size matches target"), file retained

#### Test outcome

In [ ]:
### change_raster_resolution (3): happy path - source raster deletion by default
gdgtm.change_raster_res(target_res = 500,
                        source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif',
                        dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                       )

print("Source file retained: " + Path('home/pete/Documents/tests_and_vals/gdgtm/02_ master_reprojected/chelsa_pr_reprojected.tif'))
## Expected outcome: Match confirmation ("Resolution change successful: new pixel size matches target"), file not retained

#### Test outcome

### set_raster_boundbox

In [ ]:
### set_raster_boundbox (1): failure - provide invalid bounding box

new_bb = [575000, 5235000, 565000, -5255000]
gdgtm.set_raster_boundbox(target_bb = new_bb,
                          source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                          dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/04_ master_rebound/chelsa_pr_rebound.tif',
                          delete_source = True
                         )

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif'))
## Expected outcome: Error message, file retained

#### Test outcome

In [ ]:
### set_raster_boundbox (2): happy path, retain source raster

new_bb = [555000, 5235000, 565000, 5255000]
gdgtm.set_raster_boundbox(target_bb = new_bb,
                          source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                          dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/04_ master_rebound/chelsa_pr_rebound.tif',
                          delete_source = False
                         )

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif'))
## Expected outcome: Outcome message ("Setting new bounding box not successful: errors relative to target > 0.01"), source file retained

#### Test outcome

In [ ]:
### set_raster_boundbox (3): happy path, source raster deleted by default

new_bb = [555000, 5235000, 565000, 5255000]
gdgtm.set_raster_boundbox(target_bb = new_bb,
                          source_raster = '/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif',
                          dst_raster = '/home/pete/Documents/tests_and_vals/gdgtm/04_ master_rebound/chelsa_pr_rebound.tif',
                         )

print("Source file retained: " + Path('/home/pete/Documents/tests_and_vals/gdgtm/03_ master_rescaled/chelsa_pr_rescaled.tif'))
## Expected outcome: Outcome message ("Setting new bounding box not successful: errors relative to target > 0.01"), source file not retained

#### Test outcome

## Section 3: Mosaics and Raster combinations

## Section 4: Standardization to Master

## Section 5: Raster correspondence validation

## Section 6: E2E from .json